In [1]:
import numpy as np
from scipy.sparse.linalg import gmres

In [13]:
def x_momentum(u_grid_dim, u_old, u_up_wall, v_old, p_old, u0, row, mu, g, alpha_u, delta_x, delta_y, delta_t, e):
    u_values = u_old.copy()
    v_values = v_old.copy()
    p_values = p_old.copy()

    u_rows, u_cols = u_grid_dim

    # u velocity cell coefficients
    Fe = np.zeros((u_rows - 2, u_cols))
    Fw = np.zeros((u_rows - 2, u_cols))
    Fn = np.zeros((u_rows - 2, u_cols))
    Fs = np.zeros((u_rows - 2, u_cols))
    De = np.zeros((u_rows - 2, u_cols))
    Dw = np.zeros((u_rows - 2, u_cols))
    Dn = np.zeros((u_rows - 2, u_cols))
    Ds = np.zeros((u_rows - 2, u_cols))
    aP_o = np.zeros((u_rows - 2, u_cols))
    aP = np.zeros((u_rows - 2, u_cols))
    aE = np.zeros((u_rows - 2, u_cols))
    aW = np.zeros((u_rows - 2, u_cols))
    aN = np.zeros((u_rows - 2, u_cols))
    aS = np.zeros((u_rows - 2, u_cols))
    delta_p = np.zeros((u_rows - 2, u_cols))
    d_u = np.zeros((u_rows - 2, u_cols))
    # constructing a zero matrix
    zero = np.zeros((u_rows - 2, u_cols))

    # calculating mass flow rates at staggered cell faces:
    Fe[:, -1] = 0.0
    Fe[:, :-1] = row / 2.0 * (u_values[1:-1, :-1] + u_values[1:-1, 1:])

    Fw[:, 0] = 0.0
    Fw[:, 1:] = row / 2.0 * (u_values[1:-1, :-1] + u_values[1:-1, 1:])

    Fn[0, :] = 0
    Fn[1:, :] = row / 2.0 * (v_values[1:-1, :-1] + v_values[1:-1, 1:])

    Fs[-1, :] = 0
    Fs[0:-1, :] = row / 2.0 * (v_values[1:-1, :-1] + v_values[1:-1, 1:])

    # calculating diffusion fluxes at staggered cell faces:
    De[:, -1] = 0.0
    De[:, :-1] = mu / delta_x
    Dw[:, 0] = 0.0
    Dw[:, 1:] = mu / delta_x
    Dn[0, :] = 0.0
    Dn[1:, :] = mu / delta_y
    Ds[-1, :] = 0.0
    Ds[0:-1, :] = mu / delta_y

    # pressure source term: 
    delta_p[:, :] = (p_values[1: -1, :- 1] - p_values[1: -1, 1:]) * delta_y
    # body force term + pressure source term:
    b = delta_p + row * g

    # calculating neighbouring coefficients. for a staggered cell centres:
    aE[:, :] = (De[:, :] + np.fmax(-Fe[:, :], zero[:, :])) * delta_y
    aW[:, :] = (Dw[:, :] + np.fmax(Fw[:, :], zero[:, :])) * delta_y
    aN[:, :] = (Dn[:, :] + np.fmax(-Fn[:, :], zero[:, :])) * delta_x
    aS[:, :] = (Ds[:, :] + np.fmax(Fs[:, :], zero[:, :])) * delta_x
    aP_o[:, :] = (row * delta_x * delta_y) / delta_t

    # calculating central coefficients. of staggered cells
    aP[:, :] = aP_o[:, :] + np.fmax(Fe[:, :], zero[:, :]) * delta_y + np.fmax(-Fw[:, :], zero[:, :]) * delta_y + \
               np.fmax(Fn[:, :], zero[:, :]) * delta_x + np.fmax(-Fs[:, :], zero[:, :]) * delta_x + \
               De[:, :] * delta_y + Dw[:, :] * delta_y + Dn[:, :] * delta_x + Ds[:, :] * delta_x

    n_u = 0
    # solving for u - velocity in the grid until convergence
    while True:
        n_u += 1

        # Re-arranging the equations in the form: -aNUN + aPUP - aSUS = u_source, where u_source = aEUE + aWUW + b
        # After re-arranging in the above form, equations are solved using TDMA scheme for every North-South direction
        # in the grid, sweeping from West to East.
        u_vals = u_values.copy()
        u_source = np.zeros(u_rows - 2)

        for col in np.arange(0, u_cols):

            tdma = np.diag(aP[:, col], k=0) + np.diag(-aN[1:, col], k=-1) + np.diag(-aS[:-1, col], k=1)

            if col == 0:
                u_source[:] = b[:, col] + aE[:, col] * u_values[1:-1, col + 1] + aP_o[:, col] * u0[1:-1, col] + \
                              ((1. - alpha_u) * aP[:, col] / alpha_u) * u_old[1:-1, col]
            elif col == u_cols - 1:
                u_source[:] = b[:, col] + aW[:, col] * u_values[1:-1, col - 1] + aP_o[:, col] * u0[1:-1, col] + \
                              ((1. - alpha_u) * aP[:, col] / alpha_u) * u_old[1:-1, col]
            else:
                u_source[:] = b[:, col] + aE[:, col] * u_values[1:-1, col + 1] + \
                              aW[:, col] * u_values[1:-1, col - 1] + aP_o[:, col] * u0[1:-1, col] + \
                              ((1. - alpha_u) * aP[:, col] / alpha_u) * u_old[1:-1, col]

                # implementing BC: u = 0 at left and right wall
            if col == 0 or col == u_cols - 1:
                tdma[:] = 0.0

                for i in range(u_rows - 2):
                    for j in range(u_rows - 2):
                        if i == j:
                            tdma[i, j] = 1.0

                u_source[:] = 0.0
            else:
                tdma[0, :] = 0.0
                tdma[0, 0] = 1.0
                u_source[0] = u_up_wall  # from BC on up wall........

            # solving for u momentum:
            u_solution = gmres(tdma, u_source)[0]

            # updating u values grid:
            u_values[1:-1, col] = u_solution[:]

            # BC: upper wall is moving:
            if col == 0 or col == u_cols - 1:
                u_values[0, col] = 2.0
            else:
                u_values[0, col] = u_solution[0]

            # BC: lower wall is stationary, u = 0
            u_values[-1, col] = - u_solution[-1]

        residue = abs(u_vals[1: -1, 1: -1] - u_values[1: -1, 1: -1])
        # print('************ Residue **************', '\n')
        # print(np.sum(residue), '\n')
        if np.sum(residue) < e or n_u > 1e04:
            break

    return u_values, aP

In [14]:
def y_momentum(v_grid_dim, v_old, u_old, p_old, v0, row, mu, g, alpha_v, delta_x, delta_y, delta_t, e):
    v_rows, v_cols = v_grid_dim

    u_values = u_old.copy()
    v_values = v_old.copy()
    p_values = p_old.copy()

    # v velocity cell coefficients
    Fe_ = np.zeros((v_rows, v_cols - 2))
    Fw_ = np.zeros((v_rows, v_cols - 2))
    Fn_ = np.zeros((v_rows, v_cols - 2))
    Fs_ = np.zeros((v_rows, v_cols - 2))
    De_ = np.zeros((v_rows, v_cols - 2))
    Dw_ = np.zeros((v_rows, v_cols - 2))
    Dn_ = np.zeros((v_rows, v_cols - 2))
    Ds_ = np.zeros((v_rows, v_cols - 2))
    aP_o_ = np.zeros((v_rows, v_cols - 2))
    aP_ = np.zeros((v_rows, v_cols - 2))
    aE_ = np.zeros((v_rows, v_cols - 2))
    aN_ = np.zeros((v_rows, v_cols - 2))
    aS_ = np.zeros((v_rows, v_cols - 2))
    aW_ = np.zeros((v_rows, v_cols - 2))
    delta_p_ = np.zeros((v_rows, v_cols - 2))
    d_v = np.zeros((v_rows, v_cols - 2))
    # constructing a zero matrix
    zero_ = np.zeros((v_rows, v_cols - 2))

    # calculating mass flow rates at staggered cell faces:
    Fs_[-1, :] = 0.0
    Fs_[:-1, :] = row / 2.0 * (v_values[:-1, 1:-1] + v_values[1:, 1:-1])

    Fn_[0, :] = 0.0
    Fn_[1:, :] = row / 2.0 * (v_values[:-1, 1:-1] + v_values[1:, 1:-1])

    Fw_[:, 0] = 0
    Fw_[:, 1:] = row / 2.0 * (u_values[:-1, 1:-1] + u_values[1:, 1:-1])

    Fe_[:, -1] = 0
    Fe_[:, :-1] = row / 2.0 * (u_values[:-1, 1:-1] + u_values[1:, 1:-1])

    # calculating diffusion fluxes at staggered cell faces:
    De_[:, -1] = 0.0
    De_[:, 0: -1] = mu / delta_x
    Dw_[:, 0] = 0.0
    Dw_[:, 1:] = mu / delta_x
    Dn_[0, :] = 0.0
    Dn_[1:, :] = mu / delta_y
    Ds_[-1, :] = 0.0
    Ds_[:-1, :] = mu / delta_y

    # pressure source term: 
    delta_p_[:, :] = (p_values[:-1, 1: -1] - p_values[1:, 1: -1]) * delta_y
    # body force term + pressure source term:
    b_ = delta_p_ + row * g

    # calculating neighbouring coefficients for a staggered cell centres:
    aE_[:, :] = (De_[:, :] + np.fmax(-Fe_[:, :], zero_[:, :])) * delta_y
    aW_[:, :] = (Dw_[:, :] + np.fmax(Fw_[:, :], zero_[:, :])) * delta_y
    aN_[:, :] = (Dn_[:, :] + np.fmax(-Fn_[:, :], zero_[:, :])) * delta_x
    aS_[:, :] = (Ds_[:, :] + np.fmax(Fs_[:, :], zero_[:, :])) * delta_x
    aP_o_[:, :] = (row * delta_x * delta_y) / delta_t

    # calculating central coefficients of staggered cells
    aP_[:, :] = aP_o_[:, :] + np.fmax(Fe_[:, :], zero_[:, :]) * delta_y + np.fmax(-Fw_[:, :], zero_[:, :]) * delta_y +\
                np.fmax(Fn_[:, :], zero_[:, :]) * delta_x + np.fmax(-Fs_[:, :], zero_[:, :]) * delta_x + \
                De_[:, :] * delta_y + Dw_[:, :] * delta_y + Dn_[:, :] * delta_x + Ds_[:, :] * delta_x

    # solving for v - velocity in the grid until convergence
    n_v = 0
    while True:

        n_v += 1

        # Re-arranging the equations in the form: -aEUE + aPUP - aWUW = u_source, where u_source = aNUN + aSUS + b
        # After re-arranging in the above form, equations are solved using TDMA scheme for every North-South direction
        # in the grid, sweeping from West to East.
        v_vals = v_values.copy()
        v_source = np.zeros(v_cols - 2)

        for rows in np.negative(np.arange(1, v_rows)):

            tdma_ = np.diag(aP_[rows, :], k=0) + np.diag(-aE_[rows, :-1], k=1) + np.diag(-aW_[rows, 1:], k=-1)

            if rows == 0:
                v_source[:] = b_[rows, :] + aN_[rows, :] * v_values[rows + 1, 1:-1] + aP_o_[rows, :] * v0[rows, 1:-1] +\
                              ((1. - alpha_v) * aP_[rows, :] / alpha_v) * v_old[rows, 1:-1]
            elif rows == v_rows - 1:
                v_source[:] = b_[rows, :] + aS_[rows, :] * v_values[rows - 1, 1:-1] + aP_o_[rows, :] * v0[rows, 1:-1] +\
                              ((1. - alpha_v) * aP_[rows, :] / alpha_v) * v_old[rows, 1:-1]
            else:
                v_source[:] = b_[rows, :] + aN_[rows, :] * v_values[rows + 1, 1:-1] +\
                              aS_[rows, :] * v_values[rows - 1, 1:-1] + aP_o_[rows, :] * v0[rows, 1:-1] +\
                              ((1. - alpha_v) * aP_[rows, :] / alpha_v) * v_old[rows, 1:-1]

            # BC v = 0 at up and down wall:
            if rows == -1 or rows == -v_rows:

                tdma_[:] = 0.0

                for i in range(v_cols - 2):
                    for j in range(v_cols - 2):
                        if i == j:
                            tdma_[i, j] = 1.0

                v_source[:] = 0.0

            # solving for v momentum:
            v_solution = gmres(tdma_, v_source)[0]

            # updating v values grid:
            v_values[rows, 1: -1] = v_solution[:]

            # BC: left and right wall are stationary:
            v_values[rows, 0] = - v_solution[0]
            v_values[rows, -1] = - v_solution[-1]

        residue = abs(v_vals[1: -1, 1: -1] - v_values[1: -1, 1: -1])
        # print('************ Residue **************', '\n')
        # print(np.sum(residue), '\n')
        if np.sum(residue) < e or n_v > 1e04:
            break

    return v_values, aP_

In [15]:
def pressure(p_grid_dim, aP_u, aP_v, u_values, v_values, alpha_u, alpha_v, row, delta_x, delta_y):
    p_rows, p_cols = p_grid_dim

    # pressure coefficients:
    aE_p = np.zeros((p_rows - 2, p_cols - 2))
    aW_p = np.zeros((p_rows - 2, p_cols - 2))
    aS_p = np.zeros((p_rows - 2, p_cols - 2))
    aN_p = np.zeros((p_rows - 2, p_cols - 2))
    aP_p = np.zeros((p_rows - 2, p_cols - 2))
    b_p_we = np.zeros((p_rows - 2, p_cols - 2))
    b_p_sn = np.zeros((p_rows - 2, p_cols - 2))
    b_p = np.zeros((p_rows - 2, p_cols - 2))

    # calculating d coefficients for pressure correction
    d_u = alpha_u * delta_y / aP_u
    d_v = alpha_v * delta_x / aP_v

    aE_p[:, : -1] = row * d_u[:, 1: -1] * delta_y
    aW_p[:, 1:] = row * d_u[:, 1: -1] * delta_y
    aS_p[: -1, :] = row * d_v[1: -1, :] * delta_x
    aN_p[1:, :] = row * d_v[1: -1, :] * delta_x

    aP_p[:, :] = aE_p[:, :] + aW_p[:, :] + aN_p[:, :] + aS_p[:, :]

    # pressure source term: (convergence check will be performed on this variable)
    b_p_we[:, :] = row * delta_y * (u_values[1: -1, :-1] - u_values[1: -1, 1:])
    b_p_sn[:, :] = row * delta_x * (v_values[1:, 1: -1] - v_values[: -1, 1: -1])
    b_p[:, :] = b_p_we[:, :] + b_p_sn[:, :]

    # making a copy of old p values
    p_ = np.zeros(p_grid_dim)

    # solving for pressure in the grid until convergence:
    n_p = 0
    while True:

        n_p += 1

        p_source = np.zeros(p_rows - 2)
        p_vals = p_.copy()

        for cols in range(0, p_cols):

            if 1 <= cols <= p_cols - 2:

                # constructing p_tdma
                p_tdma = np.diag(aP_p[:, cols - 1], k=0) + np.diag(-aN_p[1:, cols - 1], k=-1) + np.diag(
                    -aS_p[: -1, cols - 1], k=1)
                if cols == 1:
                    p_source[:] = b_p[:, cols - 1] + aE_p[:, cols - 1] * p_[1: -1, cols + 1]
                elif cols == p_cols - 2:
                    p_source[:] = b_p[:, cols - 1] + aW_p[:, cols - 1] * p_[1: -1, cols - 1]
                else:
                    p_source[:] = b_p[:, cols - 1] + aE_p[:, cols - 1] * p_[1: -1, cols + 1] +\
                                  aW_p[:, cols - 1] * p_[1:- 1, cols - 1]

                # from BC:
                if cols == p_cols - 2:
                    p_tdma[1, :] = 0.0
                    # p_tdma[0, 0] = 1.0
                    p_tdma[1, 1] = 1.0
                    # p_tdma[2, 2] = 1.0
                    p_source[1] = 0.0  # pressure is zero at one cell near right wall

                # solving for pressure:
                p_solution = gmres(p_tdma, p_source)[0]

                # from BC:
                if cols == 1:
                    p_[1: -1, cols - 1] = p_solution[:]
                elif cols == p_cols - 2:
                    p_[1: -1, cols + 1] = p_solution[:]

                p_[1: -1, cols] = p_solution[:]

                p_[0, cols] = p_solution[0]
                p_[-1, cols] = p_solution[-1]

        residue = abs(p_vals[1: -1, 1: -1] - p_[1: -1, 1: -1])
        # print('************ Residue **************', '\n')
        # print(np.sum(residue), '\n')
        if np.sum(residue) < 0.001 or n_p > 1e04:
            break

    return p_, b_p, d_u, d_v

In [16]:
def pressure_correction(u_n, u_old, v_n, v_old, p_n, p_old, p_grid_dim, alpha_u, alpha_v, alpha_p, d_u, d_v):
    # correcting u:
    u_cor = u_n.copy()
    u_cor[2: -1, 1: -1] += d_u[1:, 1: -1] * (p_n[2: -1, 1: -2] - p_n[2: -1, 2: -1])
    u_cor[-1, :] = - u_cor[-2, :]  # from down wall BC.

    u_new = u_cor.copy()
    u_new[2: -1, 1: -1] = alpha_u * u_cor[2: -1, 1: -1] + (1.0 - alpha_u) * u_old[2: -1, 1: -1]
    u_new[-1, :] = - u_new[-2, :]  # from down wall BC.

    # correcting v:
    v_cor = v_n.copy()
    v_cor[1: -1, 1: -1] += d_v[1: -1, 0:] * (p_old[1: -2, 1: -1] - p_old[2: -1, 1: -1])
    # from left and right wall BC:
    v_cor[:, 0] = - v_cor[:, 1]
    v_cor[:, -1] = - v_cor[:, -2]

    v_new = v_cor.copy()
    v_new[1: -1, 1: -1] = alpha_v * v_cor[1: -1, 1: -1] + (1.0 - alpha_v) * v_old[1: -1, 1: -1]
    # from left and right wall BC:
    v_new[:, 0] = - v_new[:, 1]
    v_new[:, -1] = - v_new[:, -2]

    # correcting p:
    p_new = np.zeros(p_grid_dim)
    p_new[1: -1, 1: -1] = p_old[1: -1, 1: -1] + alpha_p * p_n[1: -1, 1: -1]
    # satisfying BC for pressure:
    p_new[1: -1, 0] = p_new[1: -1, 1]
    p_new[1: -1, -1] = p_new[1: -1, -2]
    p_new[0, 1: -1] = p_new[1, 1: -1]
    p_new[-1, 1: -1] = p_new[-2, 1: -1]

    return u_new, v_new, p_new

In [17]:
def check_convergence(e, b_p, u_cor, u_old, v_cor, v_old, p_cor, p_old):
    convergence = False

    p_dim = p_cor.shape
    p_rows, p_cols = p_dim

    # checking convergence:

    # residue of u:
    residue_u = abs(u_cor[1: -1, 1: -1] - u_old[1: -1, 1: -1])
    # residue of v:
    residue_v = abs(v_cor[1: -1, 1: -1] - v_old[1: -1, 1: -1])
    # residue of p:
    residue_p = abs(p_cor[1: -1, 1: -1] - p_old[1: -1, 1: -1])

    print('*************** u- residue ****************', '\n')
    print(np.sum(residue_u), '\n')
    print('*************** v- residue ****************', '\n')
    print(np.sum(residue_v), '\n')
    print('*************** p- residue ****************', '\n')
    print(np.sum(residue_p), '\n')
    print('\n')

    if b_p[b_p < e].shape == ((p_rows - 2) * (p_cols - 2),):
        print('################ Convergence #############', '\n')
        # print('!!!!!!!!!! Time = ', t, '!!!!!!!!!!!', '\n')
        print('*************** u-velocity solution ****************', '\n')
        print(u_cor, '\n')
        print('*************** v-velocity solution ****************', '\n')
        print(v_cor, '\n')
        print('*************** pressure solution ****************', '\n')
        print(p_cor, '\n')

        convergence = True

    return convergence

############ Iteration  0  ############ 

*************** u- residue **************** 

1489.34593477 

*************** v- residue **************** 

0.0 

*************** p- residue **************** 

418302426.399 



############ Iteration  1  ############ 

*************** u- residue **************** 

11509.7409902 

*************** v- residue **************** 

123427.600199 

*************** p- residue **************** 

10720548479.3 



############ Iteration  2  ############ 

*************** u- residue **************** 

23276.6480705 

*************** v- residue **************** 

159470.247718 

*************** p- residue **************** 

1.31338004707e+12 



############ Iteration  3  ############ 



KeyboardInterrupt: 